In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('cleaned_data.csv')
data ['Date'] = data['Unnamed: 0']
data.set_index('Date', inplace=True)
data.drop(columns=['Unnamed: 0'], inplace=True)
data

,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,delta,gamma,optionid,contract_size,D to Expiration,Close,Moneyness
Date,,,,,,,,,,,,,
2000-01-03,P,1465.0,30.000,32.000,68,0.218037,-0.521459,0.005654,10526329,100,19,1455.219971,1.006721
2000-01-03,P,1475.0,35.000,37.000,914,0.212755,-0.579386,0.005687,10530656,100,19,1455.219971,1.013592
2000-01-03,P,1440.0,19.875,21.125,691,0.227943,-0.385638,0.005192,10563021,100,19,1455.219971,0.989541
2000-01-03,C,1460.0,23.375,25.375,386,0.191068,0.505087,0.006461,10949021,100,19,1455.219971,1.003285
2000-01-03,C,1450.0,29.125,32.000,1312,0.198929,0.567313,0.006117,11070856,100,19,1455.219971,0.996413
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,P,4575.0,85.900,86.300,1,0.108174,-0.606892,0.002511,156187023,100,36,4507.660156,1.014939
2023-08-31,P,4580.0,88.700,89.100,0,0.107592,-0.619895,0.002500,156269376,100,36,4507.660156,1.016048
2023-08-31,P,4585.0,91.600,92.000,6,0.107071,-0.632808,0.002485,156643075,100,36,4507.660156,1.017157


In [3]:
data['D to Expiration'].unique()

array([19, 18, 17, 16, 15, 12, 11, 10,  9, 36,  8, 32,  4, 31,  3, 30,  2,
       29, 26, 25, 24, 23, 22, 33,  5,  1,  7,  0,  6, 35, 28, 21, 14, 34,
       27, 20, 13], dtype=int64)

In [4]:
def filter_data(df):
    result = pd.DataFrame()
    for date, group in df.groupby(df.index):
        # Separate calls and puts
        calls = group[group['cp_flag'] == 'C']
        puts = group[group['cp_flag'] == 'P']
        
        # Initialize sets to keep track of used optionids
        used_optionids = set()
        
        # Define maturity groups
        maturity_groups = {
            '1_day': (0, 3),
            '1_week': (4, 14),
            '1_month': (23, 37)
        }
        
        # Find the closest moneyness to 1 for each maturity group
        for group_name, (min_days, max_days) in maturity_groups.items():
            # Filter options within the maturity group
            available_calls = calls[(calls['D to Expiration'] >= min_days) & (calls['D to Expiration'] <= max_days)]
            available_puts = puts[(puts['D to Expiration'] >= min_days) & (puts['D to Expiration'] <= max_days)]
            
            # Filter out already used optionids
            available_calls = available_calls[~available_calls['optionid'].isin(used_optionids)]
            available_puts = available_puts[~available_puts['optionid'].isin(used_optionids)]
            
            if not available_calls.empty and not available_puts.empty:
                # Select the call and put with moneyness closest to 1
                closest_call = available_calls.iloc[(available_calls['Moneyness'] - 1).abs().argsort()[:1]]
                closest_put = available_puts.iloc[(available_puts['Moneyness'] - 1).abs().argsort()[:1]]
                
                result = pd.concat([result, closest_call, closest_put])
                used_optionids.add(closest_call['optionid'].values[0])
                used_optionids.add(closest_put['optionid'].values[0])
    
    return result

# Apply the filter function
filtered_data = filter_data(data)

In [5]:
filtered_data.to_csv('filtered_data.csv')